In [1]:
    import paho.mqtt.client as mqtt
    import time
    import pandas as pd
    import csv

    def data2csv(device, location, temp, avg_temp, battery, date):
        
        df = pd.read_csv("Data_Files/TestData.csv", sep=",")

        df3 = df[['Device','Location','Temperature','Average temperature', 'Battery', 'Date']]

        df3 = df3.append({'Device': device ,'Location': location,'Temperature': temp,'Average temperature': avg_temp, 'Battery': battery, 'Date': date},                      ignore_index=True)

        display(df3)

        df3.to_csv('Data_Files/TestData.csv')
        
        
    def on_log(client, userdata, level, buf):
        print("log:", buf)

    def on_connect(client, userdata, flags, rc):
        if rc==0:
            print("Connected OK")
        else:
            print("Bad connection returned code= ",rc)

    def on_disconnect(client, userdata, flags, rc=0):
        print("Disconnected result code "+str(rc))

    def on_message(client, userdata, msg):
        topic=msg.topic
        rmsg = []
        #Decodes message and stores message in m_decode  
        m_decode=str(msg.payload.decode("utf-8","ignore"))
        print("message recieved", m_decode)
        #splits message
        rmsg = m_decode.split()
        
        #When the message contains 6 pieces of data the correct data is assigned
        if(len(rmsg) == 6):
            Device = rmsg[0]
            Location = rmsg[1]
            Temperature = rmsg[2]
            Average_temperature = rmsg[3]
            Battery = rmsg[4]
            Date = rmsg[5]
            print("Device id: " ,Device)
            print("Location: ", Location)
            print("Temp: ", Temperature)
            print("AVG temp: ", Average_temperature)
            print("Battery: ", Battery)
            print("Date: " ,Date)
            data2csv(Device, Location, Temperature, Average_temperature, Battery, Date)
        
        print(rmsg)

    broker = "test.mosquitto.org"
    client = mqtt.Client("tracing1")
    counter = 0
    client.on_connect=on_connect
    client.on_disconnect=on_disconnect
    client.on_message=on_message
    print("Connecting to broker: ", broker)
    client.connect(broker)
    client.loop_start()
    client.subscribe("tracing")
    client.publish("tracing","Hallo Server")
    while(counter<1000):
        time.sleep(1)
        counter += 1
    client.loop_stop()
    client.disconnect() 


Connecting to broker:  test.mosquitto.org
Connected OK
message recieved Hallo Server
['Hallo', 'Server']
message recieved AM1 Amsterdam 140 130 130 2021/5/20 
Device id:  AM1
Location:  Amsterdam
Temp:  140
AVG temp:  130
Battery:  130
Date:  2021/5/20


,Device,Location,Temperature,Average temperature,Battery,Date
0,AM1,Amsterdam,140,130,130,2021/5/20


['AM1', 'Amsterdam', '140', '130', '130', '2021/5/20']
